In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv',encoding = "ISO-8859-1")

In [3]:
# see data
data.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [4]:
titleSeries = data['Journal title']

In [5]:
titleSeries = titleSeries.str.lower()

In [6]:
titles = list(set(titleSeries))
len(titles)

929

In [7]:
# we want to do operations like the following
data2 = data.copy()
data2['Journal title'] = data2['Journal title'].str.lower()
data2['Count'] = 1
count = data2[['Journal title','Count']].groupby(['Journal title'], axis=0).count().sort_values(by='Count', ascending=False)

In [ ]:
# the question is whether there aren't many titles that are classified lower because they exact spelling doesn't match. 

# Some functions

In [8]:
# write function that finds all instances that starts with a certain regular expression
def find_tokens(df_split, expr):
    all_j_idx = df_split.applymap(lambda x: re.match(expr,str(x)))
    all_j_tokens = []
    shape_idx = all_j_idx.shape
    for ii in range(0,shape_idx[0]):   
        for jj in range(0, shape_idx[1]):
            if all_j_idx.iloc[ii,jj] is not None:
                all_j_tokens.append(df_split.iloc[ii,jj])
    return list(set(all_j_tokens))

In [9]:
# write function that replace certain tokens by others
def replace_tokens(df_split, to_replace, by_token):
    aux_df_split = df_split.copy()
    shape_idx = aux_df_split.shape
    for ii in range(0,shape_idx[0]):
        for jj in range(0, shape_idx[1]):
            if aux_df_split.iloc[ii,jj] in to_replace:
                aux_df_split.iloc[ii,jj] = by_token
    return aux_df_split

In [10]:
# finds titles that have a particular token in them
def find_titles(df_split, token):
    shape_idx = df_split.shape
    aux_titles = []
    for ii in range(0, shape_idx[0]):
        for jj in range(0, shape_idx[1]):
            if df_split.iloc[ii,jj] == token:
                aux_titles.append([data.iloc[ii,1], titleSeries.iloc[ii]])
    return aux_titles

# Cleaning

In [ ]:
# step 1: split the titles into words

In [11]:
titleSeries_split = titleSeries.str.split('[^a-z0-9]', expand=True)
titleSeries_split.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,psychological,medicine,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,biomacromolecules,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,j,med,chem,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,j,med,chem,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,j,org,chem,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# step 2: replace abbreviations by likely meaning (in order of frequency of particular abbreviation)

In [12]:
# find abbreviations
# find all tokens of length less than 4
abb_tf = np.array(titleSeries_split.applymap(lambda x: (x is not None) & (len(str(x))<=4) &(len(str(x))>0)))
allIdx = np.where(abb_tf==True)

In [13]:
abbTitles = []
allSmallWords = []
uniqueSmallWords = []
for ii in range(0,len(allIdx[0])):
    aux_title = titleSeries.iloc[allIdx[0][ii]]
    if aux_title not in abbTitles:
        abbTitles.append(aux_title)
    word = titleSeries_split.iloc[allIdx[0][ii], allIdx[1][ii]]
    allSmallWords.append(word)
    if word not in uniqueSmallWords:
        uniqueSmallWords.append(word)

In [14]:
# what are the most common abbreviations/small words?
swCount = np.zeros(len(uniqueSmallWords))
for word,ii in zip(uniqueSmallWords,range(0,len(uniqueSmallWords))):
    swIdx = [ii for ii in range(0, len(allSmallWords)) if allSmallWords[ii]==word]
    swCount[ii]= len(swIdx)

In [15]:
freq_df = pd.DataFrame({'Abb':uniqueSmallWords, 'Freq':list(swCount)})
freq_df.sort_values('Freq', ascending=False, inplace=True)

In [16]:
freq_df.head()

,Abb,Freq
4,of,599.0
72,plos,289.0
7,and,193.0
74,one,192.0
9,the,132.0


In [17]:
dropWords = ['of','and','the','in','for','an']

In [18]:
dropIdx = [ii for ii in range(0, freq_df.shape[0]) if freq_df.iloc[ii,0] in dropWords ]

In [19]:
dropIdx2 = freq_df.index[dropIdx]

In [20]:
freq_df.drop(dropIdx2,axis=0, inplace=True)

In [21]:
freq_df.head()

,Abb,Freq
72,plos,289.0
74,one,192.0
11,cell,82.0
18,bmc,43.0
0,j,30.0


In [22]:
# these small words we ignore for the time being because we belive they have meaning as they are(used function below to check)
ignoreWords = ['plos' ,'one','cell','bmc','open','bmj','acta','aids','pnas','stem','acs','part','care','gene','life',
               'rna','pain','law','bone','age','usa','nano','new','york','uk','cost','sex']

In [23]:
ignoreIdx = [ii for ii in range(0, freq_df.shape[0]) if freq_df.iloc[ii,0] in ignoreWords ]
ignoreIdx2 = freq_df.index[ignoreIdx]
freq_df.drop(ignoreIdx2,axis=0, inplace=True)

In [24]:
freq_df.head()

,Abb,Freq
0,j,30.0
62,embo,16.0
89,b,14.0
44,febs,14.0
2,chem,14.0


In [25]:
# we create a dictionary with abbreviations and word by which we will replace the abbreviations
# we identify the most appropriate word by visual inspection of the titles using the function <find_titles>
replaceDict = {'j':'journal',
               'embo':'embo',
               'b':'b',
               'febs':'febs',
               'chem':'chemistry',
               'biol':'biology',
               'med':'medical',
               'jnl':'journal',
               'a':'a',
               '1':'one',
               'd':'d',
               'mol':'molecular',
               'dev':'developed',
               'sci':'science',
               'an':'an',
               'int':'international'}

In [26]:
# other abbreviations identified through visual inspection of other elements
other_abbrev = {'microbiol': 'microbiology', 'immunol':'immunology', 'biochem':'biochemistry','epidemiol':'epidemiology',
               'infect':'infectious', 'cardiovasc':'cardiovascular','reson':'resonance','bioeth':'bioethics','pediatr':'prediatrics'}

In [27]:
# remove dropWords
s1 = replace_tokens(titleSeries_split, dropWords, None)

In [28]:
# replace abbreviations
s2 = s1.copy()
for abb in replaceDict.keys():
    s2 = replace_tokens(s2, [abb], replaceDict[abb])

In [29]:
# replace other abbreviations
s3 = s2.copy()
for abb in other_abbrev.keys():
    s3 = replace_tokens(s3, [abb], other_abbrev[abb])

In [30]:
# other stuff I noticed 1
s4 = replace_tokens(s3, ['jaids','(jaids)'],None)

In [31]:
s5= replace_tokens(s4, [None], '')

In [32]:
journalTitles = []
for ii in range(0, s4.shape[0]):
    aux_title = ''
    for jj in range(0, s4.shape[1]):
        if s4.iloc[ii, jj] is not None :
            aux_title = aux_title+' '+ str(s4.iloc[ii,jj])
    journalTitles.append(aux_title)

In [33]:
# other stuff I noticed 2
pnasTitles = [' pnas',' pnas proceedings national academy sciences united states america',' pnas usa', 
              ' proceedings national academy sciences pnas', ' proc natl acad sci u s a'] # replace with pnas
journalTitles2 = []
for ii in range(0, len(journalTitles)):
    if journalTitles[ii] in pnasTitles:
        journalTitles2.append(' pnas')
    else:
        journalTitles2.append(journalTitles[ii])

In [34]:
data2['Journal title']= journalTitles2

# Statistics

In [35]:
count = data2[['Journal title','Count']].groupby(['Journal title'], axis=0).count().sort_values(by='Count', ascending=False)

In [36]:
# 5 most common journals are 
count[0:5]

,Count
Journal title,
plos one,197
journal biological chemistry,64
neuroimage,29
plos pathogens,24
plos genetics,24


In [37]:
topJournals = count.index[0:5].values

In [38]:
topJournals

array([' plos one', ' journal biological chemistry', ' neuroimage',
       ' plos pathogens', ' plos genetics'], dtype=object)

In [39]:
cost = data2.iloc[:,4].copy()

In [40]:
cost2 = cost.to_frame().applymap(lambda x:float(re.sub(r'[^0-9.]','',str(x))))

In [41]:
data2['cost'] = cost2.values

In [42]:
data3 = data2.copy()
tooLargeIdx = [ii for ii in range(0, data3.shape[0]) if data3.iloc[ii,6]>=5000]

In [43]:
data3.iloc[tooLargeIdx,:] = float(np.nan)

In [44]:
# what does this do with the np.nan??????
mean_cost = data3[['Journal title','cost']].groupby('Journal title', axis=0).mean()

In [45]:
# these number are way too big for an article...
idxTopJournal = [ii for ii in range(0, mean_cost.shape[0]) if mean_cost.index[ii] in topJournals]
mean_cost.iloc[idxTopJournal,:]

,cost
Journal title,
journal biological chemistry,1385.347419
neuroimage,2215.168276
plos genetics,1643.110909
plos one,938.205556
plos pathogens,1572.866818


In [46]:
median_cost = data3[['Journal title','cost']].groupby('Journal title', axis=0).median()

In [47]:
idxTopJournal = [ii for ii in range(0, median_cost.shape[0]) if median_cost.index[ii] in topJournals]
median_cost.iloc[idxTopJournal,:]

,cost
Journal title,
journal biological chemistry,1299.74
neuroimage,2326.43
plos genetics,1712.73
plos one,897.61
plos pathogens,1600.25


In [48]:
std_cost = data3[['Journal title','cost']].groupby('Journal title', axis=0).std()

In [49]:
idxTopJournal = [ii for ii in range(0, std_cost.shape[0]) if std_cost.index[ii] in topJournals]
std_cost.iloc[idxTopJournal,:]

,cost
Journal title,
journal biological chemistry,408.563703
neuroimage,266.653947
plos genetics,153.366825
plos one,198.951604
plos pathogens,161.780891
